In [2]:
import pandas as pd

data = [
    ("Vardas1", 0, 165, 55),
    ("Vardas2", 1, 183, 95),
    ("Vardas3", 0, 171, 60),
    ("Vardas4", 1, 194, 102),
    ("Vardas5", 1, "?", 80),
    ("Vardas6", 1, 185, 90),
]

In [4]:
# pirmas
df = pd.DataFrame(data, columns=['vardas', 'lytis', 'ugis', 'svoris'])
filteredTable = df.drop(df[df['ugis'] == "?"].index)

men = filteredTable[filteredTable['lytis'] == 1]
print(f"Vidurkis {men['ugis'].mean()}")
print(f"Mediana  {men['ugis'].median()}")

Vidurkis 187.33333333333334
Mediana  185.0


In [7]:
#antra
from scipy.spatial import distance
import numpy as np

def norm(column):
    return (column - column.mean())/column.std()

def denorm(norm_var, column):
    return norm_var * column.std() + column.mean()

def metrikos_skaiciavimas(kintamieji, metrikos):
    return (1/np.sum(1/metrikos)*np.sum(kintamieji/metrikos))

norm_lytis = norm(df['lytis'])
norm_ugis = norm(filteredTable['ugis'])
norm_svoris = norm(df['svoris'])


atstumas_e = np.sqrt((norm_lytis[4] - norm_lytis)**2+(norm_svoris[4]-norm_svoris)**2)
del atstumas_e[4]

ugis_norm_e = metrikos_skaiciavimas(norm_ugis, atstumas_e)
ugis_e = denorm(ugis_norm_e, filteredTable['ugis'])
print(f"ugis euclidean - {ugis_e}")



atstumas_m = np.abs(norm_lytis[4] - norm_lytis)+np.abs(norm_svoris[4]-norm_svoris)
del atstumas_m[4]

ugis_norm_m = metrikos_skaiciavimas(norm_ugis, atstumas_m)
ugis_m = denorm(ugis_norm_m, filteredTable['ugis'])
print(f"ugis manhattan - {ugis_m}")


atstumas_c = np.amax(
                np.stack(
                    (np.abs(norm_lytis[4] - norm_lytis), np.abs(norm_svoris[4]-norm_svoris)
                ), axis=-1), 
            axis=1).tolist()

del atstumas_c[4]
atstumas_c = pd.DataFrame(atstumas_c)[0]

ugis_norm_c = metrikos_skaiciavimas(norm_ugis, atstumas_c)
ugis_c = denorm(ugis_norm_c, filteredTable['ugis'])
print(f"ugis Čebyšavo  - {ugis_c}")

ugis euclidean - 183.04910652784145
ugis manhattan - 183.81314372942035
ugis Čebyšavo  - 180.56052520951172


In [13]:
#trecias
filteredTable = filteredTable.astype({"ugis": np.int64})

avg_lyt = filteredTable['lytis'].mean()
avg_ugis = filteredTable['ugis'].mean()
avg_svoris = filteredTable['svoris'].mean()
# print(f"average lyt {avg_lyt}")
# print(f"average ugis {avg_ugis}")
# print(f"average svoris {avg_svoris}")

corr = filteredTable.corr()
# print(corr)
corr_lyt = corr['ugis'][0]
corr_svoris = corr['ugis'][2]
atkurimas_svoris = corr_svoris * (df['svoris'][4] - avg_svoris)
atkurimas_lyt = corr_lyt * (df['lytis'][4] - avg_lyt)
atsakymas = avg_ugis + (1/(abs(corr_lyt) + abs(corr_svoris)) * (atkurimas_svoris + atkurimas_lyt)) 
# print(atsakymas)
print(f"ugis Koreliacija - {atsakymas}")


ugis Koreliacija - 179.58884652676338


In [14]:
rekomendacijosRaw = [
    ("A. Klasikinė suknelė",                1, 1, 1, 0, 0, 0, 1),
    ("B. Retro suknelė",                    0, 1, 0, 0, 1, 0, 0),
    ("C. Klasikinis kostiumas su kelnėmis", 0, 0, 0, 1, 0, 1, 1),
    ("D. Klasikinis kostiumas su sijonu",   0, 0, 1, 0, 0, 0, 0),
    ("E. Kelnės",                           0, 1, 0, 1, 0, 0, 0),
    ("F. Sijonas",                          1, 0, 0, 0, 1, 1, 0)
]

rekomendacijosFull = pd.DataFrame(rekomendacijosRaw, \
    columns=['prekes', '1', '2', '3', '4', '5', '6', '7'])
rekomendacijos = rekomendacijosFull.drop("prekes", axis=1)
print(rekomendacijos)


   1  2  3  4  5  6  7
0  1  1  1  0  0  0  1
1  0  1  0  0  1  0  0
2  0  0  0  1  0  1  1
3  0  0  1  0  0  0  0
4  0  1  0  1  0  0  0
5  1  0  0  0  1  1  0


In [48]:
#ketvirtas
from scipy.spatial.distance import cdist
dist = cdist(rekomendacijos, rekomendacijos, 'euclid')
print(dist)
for i in range(len(dist)):
    dist[i][i] = np.inf

shortest_dist = []
for index, preke in enumerate(rekomendacijos["7"]):
    if preke == 1:
        shortest_dist_in_row_index = dist[index].argmin()
        shortest_dist_in_row = dist[index].min()
        row_name = rekomendacijosFull["prekes"][shortest_dist_in_row_index]
        shortest_dist.append((row_name, shortest_dist_in_row))

shortest_dist = pd.DataFrame(shortest_dist, columns=['prekes', 'atstumas'])
shortest_dist = shortest_dist.loc[shortest_dist.groupby('prekes').atstumas.idxmin()]
print(shortest_dist)

                              prekes  atstumas
0  D. Klasikinis kostiumas su sijonu  1.732051
1                          E. Kelnės  1.732051

[[0.         2.         2.23606798 1.73205081 2.         2.23606798]
 [2.         0.         2.23606798 1.73205081 1.41421356 1.73205081]
 [2.23606798 2.23606798 0.         2.         1.73205081 2.        ]
 [1.73205081 1.73205081 2.         0.         1.73205081 2.        ]
 [2.         1.41421356 1.73205081 1.73205081 0.         2.23606798]
 [2.23606798 1.73205081 2.         2.         2.23606798 0.        ]]
                              prekes  atstumas
0  D. Klasikinis kostiumas su sijonu  1.732051
1                          E. Kelnės  1.732051


In [24]:
# penkta
filmaiRaw = [
    ("Vardas1", 2, 5, 1, 5, 5, 3, 2),
    ("Vardas2", 2, 4, 1, 4, 4, 4, 3),
    ("Vardas3", 1, 3, 2, 3, 5, 3, 4),
    ("Vardas4", 3, 5, 1, 5, 5, 4, 5),
    ("Vardas5", 3, 5, 2, 3, 5, 3, 3),
    ("Vardas6", 2, 5, 1, 5, None, None, 2)
]

filmaiFull = pd.DataFrame(filmaiRaw, \
     columns=['naudotojas', 'siaubo', 'veiksmo',\
          'drama', 'nuotykiu', 'scifi', 'mitologija', 'istorinis'])
filmai = filmaiFull.drop("naudotojas", axis=1)

corr = filmai.corr()
vid = filmai.mean()
user_values = filmai.iloc[5]

scifi_corr = corr.scifi
atsakymas_scifi = vid.scifi + \
    (1/(sum(abs(scifi_corr.values)))) * \
    (
        scifi_corr[0] * (user_values[0] - vid[0]) + \
        scifi_corr[1] * (user_values[1] - vid[1]) + \
        scifi_corr[2] * (user_values[2] - vid[2]) + \
        scifi_corr[3] * (user_values[3] - vid[3]) + \
        scifi_corr[6] * (user_values[6] - vid[6])
    )

print(f"scifi -      {atsakymas_scifi}")

mito_corr = corr.mitologija
atsakymas_mito = vid.mitologija + \
    (1/(sum(abs(mito_corr.values)))) * \
    (
        mito_corr[0] * (user_values[0] - vid[0]) + \
        mito_corr[1] * (user_values[1] - vid[1]) + \
        mito_corr[2] * (user_values[2] - vid[2]) + \
        mito_corr[3] * (user_values[3] - vid[3]) + \
        mito_corr[6] * (user_values[6] - vid[6])
    )

print(f"mitologija - {atsakymas_mito}")

scifi -      4.699184752502252
mitologija - 3.4105927320950866


In [26]:
#sesta
from scipy import special

def erfc_value(array, value):
    return abs(value - array.mean())/array.std()

def anomaly_criteria(array):
    return 1/(2 * len(array))

A = np.asarray([15, 17, 16, 16, 18, 17, 15, 30, 18])
A = np.sort(A)

anomalies = True
dataset = A
while anomalies:
    anomalies = False
    for index, i in enumerate(dataset):
        if special.erfc(erfc_value(dataset, i)) < anomaly_criteria(dataset):
            dataset = np.delete(dataset, index)
            print(i)
            anomalies = True
            break

print('liko: ')
print(dataset)

30
15
15
16
16
liko: 
[17 17 18 18]


In [11]:
A = np.asarray([15, 17, 16, 16, 18, 17, 15, 30, 18])
A = np.sort(A)
q25 = np.quantile(A, 0.25)
q75 = np.quantile(A, 0.75)
# print(q25, q75)
condition1 = q25 - 1.5 * (q75 - q25)
condition2 = q75 + 1.5 * (q75 - q25)
# print(condition1, condition2)

print('isimtys: ')
anomalies = True
dataset = A
while anomalies:
    anomalies = False
    for index, i in enumerate(dataset):
        q25 = np.quantile(dataset, 0.25)
        q75 = np.quantile(dataset, 0.75)
        # print(q25, q75)

        condition1 = q25 - 1.5 * (q75 - q25)
        condition2 = q75 + 1.5 * (q75 - q25)
        # print(condition1, condition2)
        if i < condition1 or i > condition2:
            dataset = np.delete(dataset, index)
            print(i)
            anomalies = True


print('liko: ')
print(dataset)


isimtys: 
30
liko: 
[15 15 16 16 17 17 18 18]
